In [2]:
pip install tensorflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from scipy.special import gamma
from scipy.optimize import minimize

# 📌 Stap 1: Data inladen
df = pd.read_csv("acndata_sessions.csv")

# 📌 Stap 2: Data voorbereiden
df["connectionTime"] = pd.to_datetime(df["connectionTime"])
df["disconnectTime"] = pd.to_datetime(df["disconnectTime"])
df["duration"] = (df["disconnectTime"] - df["connectionTime"]).dt.total_seconds() / 60  # in minuten

# 📌 Stap 3: Feature Engineering
df["hour"] = df["connectionTime"].dt.hour  # Uur van de dag
df["weekday"] = df["connectionTime"].dt.weekday  # 0=maandag, 6=zondag
df["is_weekend"] = df["weekday"].apply(lambda x: 1 if x >= 5 else 0)  # Weekend indicator

# Selecteer relevante features
features = ["hour", "weekday", "is_weekend"]
X = df[features]
y = df["duration"]

# 📌 Stap 4: Train-test split en schaalverdeling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 📌 Stap 5: Neuraal Netwerk (MLP) trainen
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation="relu", input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1)  # Regressie-uitgang
])

model.compile(optimizer="adam", loss="mse", metrics=["mae"])
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# 📌 Stap 6: Model evalueren
y_pred = model.predict(X_test).flatten()
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f} min, MSE: {mse:.2f}, R²: {r2:.2f}")

# 📌 Stap 7: Erlang-C Wachttijdmodel
def erlang_c(n, rho):
    """ Berekening van de gemiddelde wachttijd volgens het Erlang-C model """
    erlang = ( (n * rho) ** n / (np.math.factorial(n) * (1 - rho)) ) / (
        sum(( (n * rho) ** k / np.math.factorial(k) ) for k in range(n)) + ( (n * rho) ** n / (np.math.factorial(n) * (1 - rho)) )
    )
    return erlang / (n * (1 - rho))

# Simulatie van wachttijden
n_servers = 5  # Aantal laadpalen
arrival_rate = 10  # Aantal auto's per uur
service_rate = 1 / (y_train.mean() / 60)  # Gemiddelde servicetijd in uur

rho = arrival_rate / (n_servers * service_rate)
if rho < 1:
    wait_time = erlang_c(n_servers, rho) / service_rate
    print(f"Gemiddelde wachttijd volgens Erlang-C: {wait_time:.2f} uur")
else:
    print("Waarschuwing: Wachtrij niet stabiel (rho ≥ 1)")

# 📌 Stap 8: Visualisatie van modelprestaties
plt.figure(figsize=(8, 5))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([0, max(y_test)], [0, max(y_test)], "--", color="red", label="Perfect Fit")
plt.xlabel("Werkelijke duur (min)")
plt.ylabel("Voorspelde duur (min)")
plt.title("Voorspelling vs Werkelijke duur")
plt.legend()
plt.show()

SyntaxError: invalid syntax (2094801249.py, line 1)

In [ ]:
#Test Huub